In [7]:
import torch
import numpy as np

In [47]:
w = torch.tensor([[1, 2],
                [3, 4]], dtype=torch.float)
x = torch.tensor([[1],
                [3]], dtype=torch.float, requires_grad = True)
xx = torch.tensor([[2],
                [9]], dtype=torch.float)
y = torch.matmul(w, x)
y 
print(y.grad_fn)

y.backward(x)
print(x.grad)


tensor([[10.],
        [14.]])


In [61]:
x = torch.tensor([2], dtype=torch.float, requires_grad = True)
xx  = torch.tensor([8], dtype=torch.float, requires_grad = True)

y = 2*x
y.backward(xx)
print(x.grad)

tensor([16.])


tensor([8])